### This notebook serves as the initial analysis notebook for proceeding with OCM Digital Media Assignement

In [1]:
import pandas as pd
import numpy as np
import spacy

* Running Dependencies

In [2]:
%run partA_utilities.ipynb

* Defining Mode and creating directory

In [3]:
mode = "prd"

make_mode_file(mode)

Directory for mode: prd already exists


* Defining Settings

In [4]:
settings = Settings(mode = mode)

* Data Input

In [5]:
input_data = pd.read_csv(settings.train_data_path)

* Data Info

In [6]:
input_data.head()

,url,title,content,label
0,https://www.platform.gr/life-2/fashion/75668-d...,Δες εδώ τις ολοκαίνουργιες μάσκες προστασίας α...,Οι συνεισφορές θα διατεθούν στο Κοινοτικό Ταμε...,Style&Fashion
1,https://www.protothema.gr/greece/article/96354...,"Φλώρινα: Έκρυβε χασίς, χάπια και άγνωστη ουσία...",Δείτε βίντεο Η στιγμή της ισχυρότατης έκρηξης ...,Sensitive Topics
2,https://www.vimaorthodoxias.gr/ekklisia-tis-el...,Άνω κάτω στην Εκκλησία της Ελλάδος για το συλλ...,Η απάντηση ήρθε λίγη ώρα αργότερα από τον Μητρ...,Religion&Spirituality
3,http://physics4u.gr/blog/2018/07/13/%ce%bf-%cf...,Ο σπειροειδής ραβδωτός Γαλαξίας μας,Ο Γαλαξίας μας αποτελείται κυρίως από ένα πυρή...,Science
4,https://www.tilestwra.com/pos-akougete-o-ichos...,Πώς ακούγεται ο ήχος έναρξης των Windows XP 80...,Πολλοί άνθρωποι αναγνωρίζουν αμέσως τον ήχο έν...,Viral Articles


* First things First: Let us check all of our labels

In [7]:
input_data.label.value_counts().sort_index()

label
Astrology                 45
Attractions               43
Automotive                44
Beauty                   131
Business&Finance         263
Culture                  360
Education                 45
Family&Relationships      89
Food&Drink                45
Healthy Living           181
Home&Garden               44
Politics                  45
Pop Culture               45
Religion&Spirituality     45
Science                  136
Sensitive Topics         270
Sports                   315
Style&Fashion             85
Tech&Computing           180
Travel                    45
Viral Articles            42
Name: count, dtype: int64

* Seems like this is the case, however the overall training labels seem a bit imbalanced



* There might be the case for needed approaches to handle Class Imbalance, but we will not deal with this right now for efficiency and speed of delivery of the final result



* Suggested Approaches: 


1. Use Precision / Recall / F1 as Evaluation Metrics: Accuracy is actually a bad metric when there is big class imbalance


2. Oversampling: Remove instances of the majority classes (NOT ADVISED, WE HAVE LITTLE DATA TO BEGIN WITH)


3. Undersampling: Include more instances of the minority classes


4. Use Tomek Link / SMOTE METHODS 
    (https://towardsdatascience.com/imbalanced-classification-in-python-smote-tomek-links-method-6e48dfe69bbc)



5. Data Augmentation: Replace parts of the sentence with other similar parts and create more training data, by making sure that the sentence still has a correct meaning


* Further Exploration

In [8]:
for index, row in input_data.loc[:10, :].iterrows():
    print(row['url'])
    print('_______________') 
    print(row['title'])
    print('_______________')
    print(row['content'])
    print('_______________')
    print(row['label'])
    print('________________________________________________')
    print('________________________________________________')
    print('________________________________________________')

https://www.platform.gr/life-2/fashion/75668-des-edo-tis-olokainourgies-maskes-prostasias-apo-ton-covid-19-tou-oikou-burberry-me-iconic-print
_______________
Δες εδώ τις ολοκαίνουργιες μάσκες προστασίας από τον Covid-19 του οίκου Burberry, με iconic print
_______________
Οι συνεισφορές θα διατεθούν στο Κοινοτικό Ταμείο ‘’Burberry Foundation COVID-19’’ Ο δημοφιλής  οίκος Burberry κυκλοφόρησε μια πρώτη συλλογή από μάσκες προσώπου με αντιμικροβιακή τεχνολογία,  διαθέσιμη αποκλειστικά online στο χαρακτηριστικό εμβληματικό print, που συνοδεύει τόσα χρόνια το συγκεκριμένο όνομα. Αξιοσημείωτο μάλιστα είναι και το γεγονός πως ένα ποσοστό της κάθε πώλησης θα καταβάλλεται στις συνεχείς προσπάθειες ανακούφισης των κοινοτήτων που εξακολουθούν να επηρεάζονται από την παγκόσμια πανδημία. Οι προστατευτικές μάσκες με το χαρακτηριστικό καρό μοτίβο του οίκου, κυκλοφορούν σε μπεζ και γαλάζιο χρώμα, ενθουσιάζοντας τους φανατικούς θαυμαστές του διάσημου Λονδρέζικου brand. Στην επιτυχία των νέων μασκών, οφε

* We can clearly see that there might be a connection between the site and maybe the textd after the first slash, which indicates the topic of interest.

* The idea is to break the url to site and the aforementioned slash value and make further analysis

In [9]:
list_of_site_titles = []
list_of_themes = []
for index, row in input_data.iterrows():
    url_str = row['url']
    url_suffix = re.search('\.gr|\.com|\.net', url_str).group(0)
    split_sentece = url_str.split(url_suffix)
    site_title = split_sentece[0].replace('https://', "").replace('http://', "").replace('www.', "")
    theme = split_sentece[1].split('/')[1]
    list_of_site_titles.append(site_title)
    list_of_themes.append(theme)
    print(site_title)
    print(theme)
    print('_____________________________________________________')

input_data['site_title'] = list_of_site_titles
input_data['theme'] = list_of_themes

platform
life-2
_____________________________________________________
protothema
greece
_____________________________________________________
vimaorthodoxias
ekklisia-tis-ellados
_____________________________________________________
physics4u
blog
_____________________________________________________
tilestwra
pos-akougete-o-ichos-enarxis-ton-windows-xp-800-pio-arga
_____________________________________________________
nailsmag
378827
_____________________________________________________
unboxholics
news
_____________________________________________________
queen
well-being
_____________________________________________________
enikos
society
_____________________________________________________
runningnews
item.php?id=10770
_____________________________________________________
clickatlife
theatro
_____________________________________________________
newsbomb
oikonomia
_____________________________________________________
esos
arthra
_____________________________________________________

In [10]:
labels_per_title_data = (input_data.loc[:, ["site_title" , "theme", "label"]]
                         .groupby(['site_title', 'label'])
                         .count()
                         .reset_index()
                         )

labels_per_title_data.head(50)

,site_title,label,theme
0,amna,Science,7
1,ant1news,Sensitive Topics,3
2,apetogentleman,Style&Fashion,6
3,archaiologia,Culture,39
4,arxondasbet,Sensitive Topics,13
5,astrology,Astrology,32
6,athensmagazine,Food&Drink,8
7,athensmagazine,Home&Garden,9
8,athensmagazine,Sensitive Topics,19
9,athensvoice,Culture,31


In [11]:
labels_per_theme_data = (input_data.loc[:, ["site_title" , "theme", "label"]]
                         .groupby(['theme', 'label'])
                         .count()
                         .reset_index()
                         )

labels_per_theme_data.tail(50)

,theme,label,site_title
546,telliko-meta-apo-25xronia,Sports,1
547,telos-gia-sakkari-tsitsipa,Sports,1
548,test-drive,Automotive,7
549,tha-doume-polles-ekplikseis,Sports,1
550,the-medium,Tech&Computing,1
551,the-rail-motorcyclist,Style&Fashion,1
552,theatre,Culture,21
553,theatro,Culture,29
554,theatro-h-epoxh-ths-mpekatsas-omada-xa-kinhthr...,Culture,1
555,themata-diatrofis,Healthy Living,1


* The idea is to use all of the provided columns, along with the new created ones (of course, removing the url per se and including the two newly created columns instead), in order to proceed with the text classification.